In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 
from datetime import datetime 

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [2]:
%run ../pylib/metaphone.py


**********************************************************************
File "/Users/eric/proj/virt-proj/data-project/sdrdl-data-projects/tablecloth.io/tablecloth.io-storytent-201811/pylib/metaphone.py", line 19, in __main__
Failed example:
    dm(u'Bartoš'), dm(u'Bartosz'), dm(u'Bartosch'), dm(u'Bartos')
Expected:
    (('PRT', ''), ('PRTS', 'PRTX'), ('PRTX', ''), ('PRTS', ''))
Got:
    (('PRTS', ''), ('PRTS', 'PRTX'), ('PRTX', ''), ('PRTS', ''))
**********************************************************************
1 items had failures:
   1 of   4 in __main__
***Test Failed*** 1 failures.


In [3]:
pkg = mp.jupyter.open_package()
#pkg = mp.jupyter.open_source_package()
pkg

In [4]:
ut = pkg.resource('users').dataframe()
ut['sound_key'] = ut.apply(lambda r: "{}/{}".format(r.last_dm,r.first_dm ), axis=1)
ut.head()

,clientid,form,last,first,last_dm,first_dm,sex,grade,birthday,sound_key
0,,Checkin,Putrus,Aelen,PTRS,ALN,Girl,1,2011-04-13,PTRS/ALN
1,,Checkin,Putrus,Yemen,PTRS,AMN,Boy,4,2008-05-05,PTRS/AMN
2,,Checkin,Alkass,Regina,ALKS,RJN,Girl,1,2012-08-26,ALKS/RJN
3,,Checkin,Saleh,Abdallah,SL,APTL,Boy,1,2011-03-23,SL/APTL
4,,Checkin,Saleh,Sarah,SL,SR,Girl,3,2009-04-27,SL/SR


In [5]:
cols = ['sound_key', 'first','last', 'first_dm','last_dm','birthday']
user_source =  ut[cols].copy()

user_source.head().T

,0,1,2,3,4
sound_key,PTRS/ALN,PTRS/AMN,ALKS/RJN,SL/APTL,SL/SR
first,Aelen,Yemen,Regina,Abdallah,Sarah
last,Putrus,Putrus,Alkass,Saleh,Saleh
first_dm,ALN,AMN,RJN,APTL,SR
last_dm,PTRS,PTRS,ALKS,SL,SL
birthday,2011-04-13,2008-05-05,2012-08-26,2011-03-23,2009-04-27


In [6]:
# The original source has a lot of dates represented as Excel date integers. 
from datetime import timedelta, datetime
from dateutil.parser import parse
def convt_date(v):
    if isinstance(v, str):
        
        try:
            return parse(v).date()
        except ValueError:
            pass
        
        try:
            days = int(v)
            if days > 1000:
                return (pd.Timestamp('1900-01-01')+pd.Timedelta(days=days)).date()
            else:
                return None
        except ValueError:
            pass
        
        return None

    else:
        try:
            return v.date()
        except AttributeError:
            return v

user_source['birthday'] = user_source.birthday.apply(convt_date)

# String version, so some types of aggregation work
user_source['birthday_str'] = user_source.birthday.apply(lambda x: str(x) if x else x)
user_source.head()

,sound_key,first,last,first_dm,last_dm,birthday,birthday_str
0,PTRS/ALN,Aelen,Putrus,ALN,PTRS,2011-04-13,2011-04-13
1,PTRS/AMN,Yemen,Putrus,AMN,PTRS,2008-05-05,2008-05-05
2,ALKS/RJN,Regina,Alkass,RJN,ALKS,2012-08-26,2012-08-26
3,SL/APTL,Abdallah,Saleh,APTL,SL,2011-03-23,2011-03-23
4,SL/SR,Sarah,Saleh,SR,SL,2009-04-27,2009-04-27


In [7]:
t = user_source[user_source.birthday.notnull()][['sound_key', 'first','last', 'first_dm','last_dm','birthday', 'birthday_str']]\
    .set_index('sound_key').drop_duplicates().copy()

In [8]:
t.head()

,first,last,first_dm,last_dm,birthday,birthday_str
sound_key,,,,,,
PTRS/ALN,Aelen,Putrus,ALN,PTRS,2011-04-13,2011-04-13
PTRS/AMN,Yemen,Putrus,AMN,PTRS,2008-05-05,2008-05-05
ALKS/RJN,Regina,Alkass,RJN,ALKS,2012-08-26,2012-08-26
SL/APTL,Abdallah,Saleh,APTL,SL,2011-03-23,2011-03-23
SL/SR,Sarah,Saleh,SR,SL,2009-04-27,2009-04-27


In [9]:

g = t.groupby('sound_key')
t['allocated']  = False

# Immediately allocate everything
for idx, group in g:
   
    if group.count()['birthday_str'] == 1:
        try:
            t.loc[idx,'allocated'] = True
        except Exception as e:
            print(e)

          
ua = t[~t.allocated]

In [10]:
first_bd  = ua.groupby('sound_key').min().reset_index().set_index(['sound_key', 'birthday']).copy()
first_bd['allocated'] = True
first_bd.head()

t = t.reset_index().set_index(['sound_key', 'birthday']).join(first_bd['allocated'], rsuffix='_other')
t['allocated'] = t.apply(lambda r: True if r.allocated_other == True else r.allocated, axis = 1)

t.drop(columns='allocated_other', inplace = True)

t.sort_values(['sound_key']).to_csv('user_table.csv')

marked = t.reset_index().copy()
marked.drop_duplicates(subset=['sound_key','birthday_str'], inplace=True)

marked['user_no'] = marked.reset_index().index

allocated =  marked[marked.allocated]

users = allocated.reset_index()[['first','last','birthday','sound_key','user_no']]
users.head()


,first,last,birthday,sound_key,user_no
0,Henry,H,2018-05-01,/HNR,0
1,Harper,H,2018-05-01,/HRPR,1
2,Jayden,1,2018-09-04,/JTN,2
3,Collin,H,2018-12-04,/KLN,3
4,Margaux,1,2017-09-04,/MRK,4


In [11]:

s = pkg.resource('combined_source').dataframe()
s.columns = ['#',
 'location',
 "first",
 "last",
 "sex",
 'grade',
 'birthday',
 'Which emoji describes how you feel about reading right now?',
 'start_date',
 'submit_date',
 'Network ID',
 'Form',
 'ClientID',
 'Tablecloth Link',
 'QR Code Link']

s['first'] = s['first'].str.strip().str.title()
s['last'] = s['last'].str.strip().str.title()

s['sound_key'] = s.apply(lambda r: "{}/{}".format(dm(r['last'])[0], dm(r['first'])[0]), axis=1)

t = s.merge(users, on='sound_key', suffixes=('','_r'), how='left')

merged = t[['sound_key', 'user_no', 'last','last_r','first','first_r','birthday','birthday_r',
            'location','sex','grade','start_date','submit_date']].copy()

def mismatch(r):
    f1, f2 = r['first'], r['first_r']
    l1, l2 = r['last'], r['last_r'] 
    
    if f1=='' and l1 =='':
        return 'empty'
    
    if l1!=l2 and f1!=f2:
        return 'both'
    elif l1!=l2:
        return 'last'
    elif f1!=f2:
        return 'first'
    else:
        return ''
    

merged['mismatch'] = merged.apply(mismatch, axis = 1)

merged = merged[merged.mismatch!='empty'].copy().sort_values(['last_r','first_r'])

merged.loc[merged.user_no.isna(),'mismatch'] = 'sound_key'

merged.fillna('', inplace=True)

merged.head()

,sound_key,user_no,last,last_r,first,first_r,birthday,birthday_r,location,sex,grade,start_date,submit_date,mismatch
5906,/TRN,9,1,1,Darian,Darian,,2018-09-04,WaypointPublic,Boy,3,2018-09-05 01:53:54,2018-09-05 01:53:54,
6839,/TRN,9,1,1,Darian,Darian,9/4/2018,2018-09-04,Waypoint Public,Boy,3,9/5/2018 1:52:46,2018-09-05 01:53:54,
5894,/JTN,2,1,1,Jayden,Jayden,,2018-09-04,WaypointPublic,Boy,1,2018-09-05 00:25:27,2018-09-05 00:25:27,
5898,/JTN,2,1,1,Jayden,Jayden,,2018-09-04,,,,,2018-09-05 00:51:18,
6848,/JTN,2,1,1,Jayden,Jayden,9/4/2018,2018-09-04,Waypoint Public,Boy,1,9/5/2018 0:24:05,2018-09-05 00:25:27,


In [12]:
locations = merged[['sound_key','location']].copy()
print(len(locations))
locations = locations[locations['location'].str.strip() != '']
print(len(locations))
locations['n'] = 1
t =locations.drop_duplicates().groupby(['sound_key']).count()
t = t.sort_values('n',ascending=False)
t = t[t.n > 1]
t = merged[merged.sound_key.isin(list(t.index))].sort_values('sound_key')
t.to_csv('multi_location.csv')

7897
6683


In [13]:
merged.head()

,sound_key,user_no,last,last_r,first,first_r,birthday,birthday_r,location,sex,grade,start_date,submit_date,mismatch
5906,/TRN,9,1,1,Darian,Darian,,2018-09-04,WaypointPublic,Boy,3,2018-09-05 01:53:54,2018-09-05 01:53:54,
6839,/TRN,9,1,1,Darian,Darian,9/4/2018,2018-09-04,Waypoint Public,Boy,3,9/5/2018 1:52:46,2018-09-05 01:53:54,
5894,/JTN,2,1,1,Jayden,Jayden,,2018-09-04,WaypointPublic,Boy,1,2018-09-05 00:25:27,2018-09-05 00:25:27,
5898,/JTN,2,1,1,Jayden,Jayden,,2018-09-04,,,,,2018-09-05 00:51:18,
6848,/JTN,2,1,1,Jayden,Jayden,9/4/2018,2018-09-04,Waypoint Public,Boy,1,9/5/2018 0:24:05,2018-09-05 00:25:27,


In [14]:
t = merged[['sound_key','last_r','first_r','birthday_r','location','sex','grade']]
t = t[ ((t['last_r'] != '') & (t.first_r != '')) ]
t = t[ (~t['last_r'].isnull()) & (~t['first_r'].isnull()) ]
t.columns = ['sound_key','last_name', 'first_name', 'birthday', 'mode_location', 'sex', 'grade']
t.head()

,sound_key,last_name,first_name,birthday,mode_location,sex,grade
5906,/TRN,1,Darian,2018-09-04,WaypointPublic,Boy,3
6839,/TRN,1,Darian,2018-09-04,Waypoint Public,Boy,3
5894,/JTN,1,Jayden,2018-09-04,WaypointPublic,Boy,1
5898,/JTN,1,Jayden,2018-09-04,,,
6848,/JTN,1,Jayden,2018-09-04,Waypoint Public,Boy,1


In [15]:
def most(x):
    """Select the most common item that isn't empty"""
    try:
        return list(e for e in x.value_counts().index if e and e.strip())[0]
    except AttributeError:
        return list(e for e in x.value_counts().index if e)[0]
    except IndexError:
        return None


t2 = t.groupby(['sound_key']).agg(
    {
        'last_name': most,
        'first_name': most,
        'birthday': 'min',
        'mode_location': most,
        'sex': most, 
        'grade': 'max'
        
    }
)
t2['grade'] = t.groupby(['sound_key']).grade.max()

def coerce_to_int(v):
    try:
        return int(v)
    except ValueError:
        return None
        

t2['grade'] = t2.grade.apply(coerce_to_int)


t2['birthday'] = pd.to_datetime(t2.birthday, errors='coerce')

t2['b_error'] = ''

t2['birthday_orig'] = t2.birthday

def birthday_error(r):

    if not r.birthday:
        return 'no_bd'

    if r.grade and r.grade > 0:
        if r.birthday > datetime(2018-5-int(r.grade), 9, 1):
            return 'too_young_for_grade'
    else:
        if r.birthday.year > 2016:
            return 'too_young'

    return ''
        

t2['bd_error'] = t2.apply(birthday_error, axis=1)

t2['birthday'] = t2.birthday.where(t2.bd_error == '', None)
t2 = t2.reset_index()
t2.insert(0, 'user_id', t2.reset_index().index+1)

user_table=t2
user_table.head(20)

,user_id,sound_key,last_name,first_name,birthday,mode_location,sex,grade,b_error,birthday_orig,bd_error
0,1,/HNR,H,Henry,NaT,WaypointPublic,Boy,0,,2018-05-01,too_young
1,2,/HRPR,H,Harper,NaT,WaypointPublic,Girl,0,,2018-05-01,too_young
2,3,/JTN,1,Jayden,NaT,Waypoint Public,Boy,1,,2018-09-04,too_young_for_grade
3,4,/KLN,H,Collin,NaT,Waypoint Public,Boy,2,,2018-12-04,too_young_for_grade
4,5,/MRK,1,Margaux,NaT,WaypointPublic,Girl,1,,2017-09-04,too_young_for_grade
5,6,/N0N,H,Nathan,NaT,Waypoint Public,Boy,0,,2018-12-04,too_young
6,7,/PRL,W,Briel,2008-02-20,SpecialStoryTents,Girl,1,,2008-02-20,
7,8,/RT,H,Reid,NaT,SpecialStoryTents,Boy,0,,2018-05-06,too_young
8,9,/TLR,H,Taylor,NaT,Waypoint Public,Girl,0,,2018-12-04,too_young
9,10,/TRN,1,Darian,NaT,WaypointPublic,Boy,3,,2018-09-04,too_young_for_grade


In [16]:
len(user_table)

1213